In [4]:
from numpy import *
from numpy.random import multinomial, random
from rpy2 import robjects
import rpy2.robjects.numpy2ri
robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
deseq = importr('DESeq2')

# Generate some data. 1000 genes, 10 samples
n = 1000
probabilities = random(n)
probabilities /= sum(probabilities)
data = zeros((n,10), int)
for i in range(10):
    data[:,i] = multinomial(1000000, probabilities)

# Make the data frame
d = {}
categories = ('1','2') * 5
d["key_1"] = robjects.IntVector(categories)
dataframe = robjects.DataFrame(d)

In [5]:
# Create the design matrix, and run DESeqDataSetFromMatrix
design = robjects.Formula("~ 1") # <--- I guess this is wrong
dds = deseq.DESeqDataSetFromMatrix(countData=data, colData=dataframe,design=design)

In [13]:
dds.

<rpy2.rinterface_lib.sexp.StrSexpVector object at 0x7f02181b1300> [RTYPES.STRSXP]